In [ ]:
# Necessary Imports
import re
import pandas as pd

In [ ]:
# Reading the data
data=pd.read_csv("data.csv")

# Pre-Processing

In [ ]:
# Function to handle price ranges
def process_price(price):
    if 'to' in price:
        prices = re.findall(r'\d+\.\d+', price)
        return (float(prices[0]) + float(prices[1])) / 2
    else:
        return float(price.replace('$', ''))

# Apply the function to the 'Price' column
data['Price'] = data['Price'].apply(process_price)

# Convert 'Star-Rating' to numeric (extract the first number before 'out of 5 stars' and convert to float)
data['Star-Rating'] = data['Star-Rating'].str.extract('(\d+\.\d+)').astype(float)

# Extract numeric values from 'Sales' (extract the first number and convert to integer)
data['Sales'] = data['Sales'].str.extract('(\d+)').fillna(0).astype(int)

# Fill missing 'Best-Seller' with a default value 'No'
data['Best-Seller'] = data['Best-Seller'].replace('', 'No')

# Extract numeric values from 'Ratings-Count' (extract the first number and convert to integer)
data['Ratings-Count'] = data['Ratings-Count'].str.extract('(\d+)').fillna(0).astype(int)


In [ ]:
# Printing the Data
data

In [ ]:
data['Star-Rating'].unique()  

## Store data into DB

In [ ]:

import sqlite3
# Connect to SQLite database (it will create the database if it does not exist)
conn = sqlite3.connect('books_data.db')
cursor = conn.cursor()


In [ ]:
# Create the table if it does not exist
create_table_query = '''
CREATE TABLE IF NOT EXISTS books (
    Title TEXT,
    Price REAL,
    Star_Rating REAL,
    Sales INTEGER,
    Status TEXT,
    Best_Seller TEXT,
    Ratings_Count INTEGER,
    Author TEXT
)
'''
cursor.execute(create_table_query)


In [ ]:
# Insert the data into the table
for index, row in data.iterrows():
    cursor.execute('''
    INSERT INTO books (Title, Price, Star_Rating, Sales, Status, Best_Seller, Ratings_Count, Author)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Data has been successfully inserted into the database.")

In [ ]:
# Printing some data from the database
conn = sqlite3.connect('books_data.db')
cursor = conn.cursor()

cursor.execute('''SELECT * FROM books LIMIT 5''')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the Connection
conn.close()